In [1]:
import pandas as pd
import os
from utils import run_reduced_scenarios, plot_uncertainty, nb_days

### What this script does ?

- Get hydrogeneration years
- Apply rules such as seasonal average to represent reservoirs management
- Find multiple representative years to be used in the optimization model
- Compute with capacity to calculate pAvailability in EPM format
- Export in .csv format

In [2]:
# All the inputs and output will be stored in the following folder
main_folder = 'data/guinea/repr_yrs'

In [3]:
rename_hpp = {
    'amaria': 'Amaria',
    'baneah': 'Baneah', 
    'bonkon_diaria': 'Bonkon-Diaria', 
    'boureya': 'Boureya', 
    'diallol': 'Diallol', 
    'diareguela': 'Diareguela', 
    'digan': 'Digan',
    'donkea': 'Donkea', 
    'farankonedou': 'Farankonedou', 
    'fello_sounga': 'Fello Sounga',
    'fetore': 'Fetore', 
    'fomi': 'Fomi', 
    'garafiri': 'Garafiri',
    'grand_chutes': 'Grand Chutes', 
    'grand_kinkon': 'Grand Kinkon',
    'guozoguezia': 'Guozoguezia',
    'hakkunde': 'Hakkunde-Mitti',
    'kaleta': 'Kaleta',
    'kassa': 'Kassa',
    'kogebedou': 'Kogebedou',
    'korafindi': 'Korafindi',
    'kouloutamba': 'Koukoutamba',
    'kouravel': 'Kouravel',
    'morisanako': 'Morisanako',
    'niagara': 'Niagara',
    'nzebela': 'Nzebela',
    'poudalde': 'Poudalde',
    'souapiti': 'Souapiti',
    'tiopo_105': 'Tiopo'
    
}

In [4]:
def calculate_seasonal_average(df):
    """Calculate the seasonal average of the data.
    
    The seasonal average is calculated for the period from November to June.
    
    Parameters
    ----------
    df : pd.DataFrame
        The data to calculate the seasonal average. The data should be in the format of a DataFrame with the columns
        'year' and 'month' and the values to calculate the seasonal average.
    
    Returns
    -------
    pd.DataFrame
    """

    temp = df.stack()
    temp.index.names = ['month', 'year']
    temp = temp.reorder_levels(['year', 'month'])
    temp = temp.sort_index()
    temp = temp.reset_index(name='value')
    temp['seasonal year'] = temp['year'].astype(int)
    temp.loc[temp['month'] >= 11, 'seasonal year'] += 1
    temp_filtered = temp[temp['month'].isin([11, 12, 1, 2, 3, 4, 5, 6])]
    seasonal_averages = temp_filtered.groupby('seasonal year')['value'].mean()
    seasonal_averages.name = 'value avg'
    temp_filtered = pd.merge(temp_filtered, seasonal_averages, left_on='seasonal year', right_index=True)
    temp_filtered.loc[:, 'value'] = temp_filtered.loc[:, 'value avg']
    temp_filtered = temp_filtered.loc[:, ['year', 'month', 'value']]
    temp_filtered.rename(columns={'value': 'value avg'}, inplace=True)

    temp = pd.merge(temp, temp_filtered, on=['year', 'month'], how='left')
    # Replace the value by the average value if value avg is not missing
    temp.loc[~temp['value avg'].isnull(), 'value'] = temp.loc[~temp['value avg'].isnull(), 'value avg']
    
    temp = temp.loc[:, ['year', 'month', 'value']]
    temp = temp.pivot(index='month', columns='year', values='value') 

    return temp

In [5]:
# Read all csv file in the folder
n_clusters, method = 3, "real"
rename_scenario = None

folder = 'data_hydro_guinea_halfreservoir'
out_folder = 'output_{}'.format(folder)

folder = os.path.join(main_folder, folder)

impact_cc = None
existing_plant = ['souapiti', 'kaleta', 'garafiri']
extract_min = True
if impact_cc is not None:
    out_folder = out_folder + '_cc'
out_folder = os.path.join(main_folder, out_folder)
if not os.path.exists(out_folder):
    os.makedirs(out_folder)
files = [f for f in os.listdir(folder) if f.endswith('.csv')]
data, results = {}, {}
tot, tot_existing = [], []

if impact_cc is not None:
    impact_cc = pd.read_csv(os.path.join(main_folder, 'impact_cc_hydro_guinea.csv'), index_col=[0]).squeeze()

# Results is not used here as reprensentative years is determine based on total hydropower production
for f in files:
    print(f)
    name = f.split('.')[0]
    df = pd.read_csv(os.path.join(folder, f), index_col=[0])
    
    # only select years after 1980
    df.columns = pd.to_numeric(df.columns)
    df = df.loc[:, [i for i in df.columns if i >= 1980]]
    df.columns = df.columns.astype(str)

    # keep the average production for the dry season
    df = calculate_seasonal_average(df)
    
    if impact_cc is not None:
        df_cc = pd.DataFrame()
        for k, i in impact_cc.items():
            temp = df.copy() * (1 + i)
            temp.columns = ['{}-{}'.format(i, k)  for i in temp.columns]
            df_cc = pd.concat([df_cc, temp], axis=1)
            
        df = df_cc.copy()
    
    # transform from mw to gwh
    # df = (df.T * nb_days).T * 24 / 1e3
    data.update({name: df.copy()})
    # results.update({f.split('.')[0]: run_reduced_scenarios(df, n_clusters=n_clusters, method=method)})
    tot = tot + [df]
    if name in existing_plant:
        print(name)
        tot_existing = tot_existing + [df]
        
# Sum all the hydropower production
tot = sum(tot)
tot_existing = sum(tot_existing)

# indicator
# average monthly capacity (MW)
prod = (tot.T * nb_days).T * 24 / 1e3
indicator = {'yearly_prod_avg': prod.sum().mean(), 'monthly_prod_avg': prod.mean(axis=1)}

# Find the representative years
if extract_min:
    min_prod_year = tot.sum().idxmin()
    # Remove the year with the minimum production
    min_prod = {k: i.loc[:, min_prod_year] for k, i in data.items()}
    data = {k: i.drop(columns=[min_prod_year]) for k, i in data.items()}
    proba_min = 1 / len(tot.columns)
    
years_repr = run_reduced_scenarios(tot, n_clusters=n_clusters, method=method)
scenarios = [i.split(' - ')[0] for i in years_repr.columns]

# Select the data for the representative years
data = {k: i.loc[:, scenarios].stack() for k, i in data.items()}
data = pd.concat(data, axis=1, names=['hpp']).T
data.columns.names = ['Month', 'Scenarios']
data = data.reorder_levels(['Scenarios', 'Month'], axis=1)
data = data.sort_index(axis=1)

# Assing the scenarios to low, medium and high
if n_clusters == 1:
    rename_scenario = {scenarios[0]: 'baseline'}
    data.rename(columns=rename_scenario, level='Scenarios', inplace=True)
elif n_clusters == 3:
    temp = tot.loc[:, scenarios].sum()
    rename_scenario = {temp.idxmin(): 'low', temp.idxmax(): 'high', [i for i in temp.index if i not in [temp.idxmin(), temp.idxmax()]][0]: 'medium'}
    data = data.rename(columns=rename_scenario, level='Scenarios')

if extract_min:
    min_prod = pd.concat(min_prod, axis=1, names=['hpp']).T
    min_prod = pd.concat([min_prod], axis=1, keys=['min'])
    min_prod.columns.names = ['Scenarios', 'Month']
    min_prod = min_prod.sort_index(axis=1)
    
    data = pd.concat([data, min_prod], axis=1)

# Rename the hpp
if rename_hpp is not None:
    data.rename(index=rename_hpp, level='hpp', inplace=True)
    
capacity_file = 'capacity_hydro_guinea.csv'
# Normalize by capacity to get the availability
if capacity_file in os.listdir(main_folder):
    capacity_file = os.path.join(main_folder, capacity_file)

    print('Capacity file exist to calculate the availability')
    capacity = pd.read_csv(capacity_file, index_col=[0]).squeeze()
    capacity = capacity.loc[data.index.get_level_values('hpp').unique()]
    availability = (data.T / capacity).T
else:
    raise ValueError('Capacity file is missing')

# Export in EPM format

availability.index.names = [None]
# data.columns.names = [None, None]
availability.round(3).to_csv(os.path.join(out_folder, 'pAvailability_hydro_{}.csv'.format(n_clusters)))

# For sensitivity analysis
if n_clusters == 3:
    for i in ['low', 'medium', 'high']:
        availability.loc(axis=1)[i].round(3).to_csv(os.path.join(out_folder, 'pAvailability_hydro_{}.csv'.format(i)))

# Format probabilities
pProbaScenarios = pd.Series([i.split(' - ')[1] for i in years_repr.columns], index=pd.Index([i.split(' - ')[0] for i in years_repr.columns], name='Scenarios'), name='Value')
if rename_scenario is not None:
    pProbaScenarios = pProbaScenarios.rename(index=rename_scenario)
if extract_min:
    pProbaScenarios.loc['min'] = proba_min
    
pProbaScenarios.to_csv(os.path.join(out_folder, 'pProbaScenarios_hydro_{}.csv'.format(n_clusters)))



fello_sounga.csv
farankonedou.csv
baneah.csv
poudalde.csv
nzebela.csv
tiopo_105.csv
niagara.csv
digan.csv
bonkon_diaria.csv
grand_chutes.csv
diallol.csv
amaria.csv
fetore.csv
kaleta.csv
kaleta
kouloutamba.csv
donkea.csv
fomi.csv
garafiri.csv
garafiri
guozoguezia.csv
hakkunde.csv
korafindi.csv
souapiti.csv
souapiti
kogebedou.csv
diareguela.csv
kouravel.csv
kassa.csv
morisanako.csv
grand_kinkon.csv
boureya.csv


/var/folders/p9/3r4_fgzd72j7b469xxshgfnh0000gn/T/ipykernel_26565/1346846648.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[10.    10.    10.    10.    10.    10.    10.25  10.25  10.25  10.25
 10.25  10.25  10.25  10.25  12.375 12.375 12.375 12.375 12.375 12.375
 12.375 12.375 12.125 12.125 12.125 12.125 12.125 12.125 12.125 12.125
 11.25  11.25  11.25  11.25  11.25  11.25  11.25  11.25  11.125 11.125
 11.125 11.125 11.125 11.125 11.125 11.125  9.375  9.375  9.375  9.375
  9.375  9.375  9.375  9.375 10.    10.    10.    10.    10.    10.
 10.    10.     9.     9.     9.     9.     9.     9.     9.     9.
  9.625  9.625  9.625  9.625  9.625  9.625  9.625  9.625 10.375 10.375
 10.375 10.375 10.375 10.375 10.375 10.375  9.125  9.125  9.125  9.125
  9.125  9.125  9.125  9.125 10.875 10.875 10.875 10.875 10.875 10.875
 10.875 10.875 11.    11.    11.    11.    11.    11.    11.    11.
 11.75  11.75  11.75  11.75 

Capacity file exist to calculate the availability


## Checking the results

In [6]:
if 'baseline' in data.columns.get_level_values('Scenarios'):
    capacity_baseline = data.xs('baseline', level='Scenarios', axis=1)
    prod = (capacity_baseline * nb_days).T * 24 / 1e3
    display(prod.sum(axis=1), prod.sum().sum())

    prod_current = prod.loc[:, ['Kaleta', 'Souapiti']]
    display(prod_current.sum(axis=1))

In [7]:
prod = data.mul(nb_days, level='Month', axis=1) * 24 / 1e3
capacity = data.sum()
existing_plant = [rename_hpp[i] for i in existing_plant if i in rename_hpp]
capacity_existing = data.loc[existing_plant, :].sum()
prod = prod.sum()
prod, capacity, capacity_existing = prod.unstack('Scenarios'), capacity.unstack('Scenarios'), capacity_existing.unstack('Scenarios')

In [8]:
filename = os.path.join(out_folder, 'production_hydro_tot_{}.png'.format(n_clusters))
plot_uncertainty(tot, df2=capacity, title="Hydropower capacity (MW)", ylabel="MW", filename=filename, ymax=4500)

filename = os.path.join(out_folder, 'production_hydro_existing_{}.png'.format(n_clusters))
plot_uncertainty(tot_existing, df2=capacity_existing, title="Hydropower capacity existing (MW)", ylabel="MW", filename=filename, ymax=1100)


In [9]:
# make_simple_plot(prod, 'Hydropower production', 'GWh')
# make_simple_plot(capacity, 'Hydropower capacity', 'MW')

In [10]:
prod

Scenarios,high,low,medium,min
Month,,,,
1,834.364134,665.412265,722.158745,617.976000
2,753.619218,601.017530,652.272415,558.171871
3,834.364134,665.412265,722.158745,617.976000
4,807.449162,643.947353,698.863302,598.041290
5,834.364134,665.412265,722.158745,617.976000
6,807.449162,643.947353,698.863302,598.041290
7,1070.960941,1005.403459,977.459521,744.367715
8,1376.374951,1093.918451,1109.850010,872.107670
9,1950.767740,1348.433322,1413.365417,1288.330324
